In [3]:
%%capture
!pip install pyheif
!pip install ExifRead

import torch
import numpy as np
import pandas as pd
import fileinput
import os
import pyheif 
import io
import exifread

from PIL import Image, ImageDraw
from pathlib import Path
from typing import Union
from copy import deepcopy
from tqdm.notebook import tqdm
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import r2_score

In [4]:
if not os.path.isfile('weights.pt'):
    weights_url = 'https://archive.org/download/anpr_weights/weights.pt'
    os.system(f'wget {weights_url}')

In [5]:
def prepend_text(filename: Union[str, Path], text: str):
    with fileinput.input(filename, inplace=True) as file:
        for line in file:
            if file.isfirstline():
                print(text)
            print(line, end="")

In [6]:
if not os.path.isdir('yolov7'):
    yolov7_repo_url = 'https://github.com/WongKinYiu/yolov7'
    os.system(f'git clone {yolov7_repo_url}')
    # Fix import errors
    for file in ['yolov7/models/common.py', 'yolov7/models/experimental.py', 'yolov7/models/yolo.py', 'yolov7/utils/datasets.py']:
         prepend_text(file, "import sys\nsys.path.insert(0, './yolov7')")

In [7]:
from yolov7.models.experimental import attempt_load
from yolov7.utils.general import check_img_size
from yolov7.utils.torch_utils import select_device, TracedModel
from yolov7.utils.datasets import letterbox
from yolov7.utils.general import non_max_suppression, scale_coords
from yolov7.utils.plots import plot_one_box

In [8]:
weights = 'weights.pt'
device_id = '0'
image_size = 640
trace = True
device = select_device(device_id)
model = attempt_load(weights, map_location=device)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


In [9]:
def detect_plate(img):
    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
        
    with torch.no_grad():
        # Inference
        pred = model(img, augment=True)[0]

    # Apply NMS
    pred = non_max_suppression(pred, 0.25, 0.45, classes=0, agnostic=True)

    plate_detections = []
    det_confidences = []
    
    # Process detections
    for i, det in enumerate(pred):  # detections per image
        if len(det):
            # Return results
            for *xyxy, conf, cls in reversed(det):
                coords = [position for position in (torch.tensor(xyxy).view(1, 4)).tolist()[0]]
                plate_detections.append(coords)
                det_confidences.append(conf.item())

    return plate_detections, det_confidences

In [10]:
def crop(image, coord):
    cropped_image = image[int(coord[1]):int(coord[3]), int(coord[0]):int(coord[2])]
    return cropped_image

In [11]:
def get_plates_from_image(input):
    if input is None:
        return None
    plate_detections, _ = detect_plate(input)
    detected_image = deepcopy(input)
    for coords in plate_detections:
        plate_region = crop(input, coords)
        plot_one_box(coords, detected_image, color=[0, 150, 255], line_thickness=2)
    return detected_image, plate_detections

In [12]:
test_img_names = os.listdir('/test/')
train_img_names = os.listdir('/train/')

train_labels_df = pd.read_csv('/train.csv', sep=';', index_col=None)
train_labels_names = train_labels_df['image_name'].values

In [13]:
def resize_download(img_names, path_img):
    data = []

    img_size = 640
    stride = 32

    for img_name in tqdm(img_names): 

        if 'heic' in img_name:
            heif_file = pyheif.read(os.path.join(path_img, img_name))
            img = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)
        else:
            img = Image.open(os.path.join(path_img, img_name))
            
        img = letterbox(np.array(img), img_size, stride=stride)[0]
        data.append(img)
    return data    

In [14]:
train_data = resize_download(train_img_names, '/train/')

  0%|          | 0/530 [00:00<?, ?it/s]

In [15]:
test_data = resize_download(test_img_names, '/test/')

  0%|          | 0/521 [00:00<?, ?it/s]

In [16]:
def central_bbox(results, wigth_half):
    wigth = [i[2] - i[0] for i in results]
    height = [i[3] - i[1] for i in results]
    ratio = np.array(wigth) / np.array(height)
    center = [np.abs((i[2] + i[0]) / 2 - wigth_half) for i in results]
    for i in range(len(center)):
        if ratio[i] < 2 or ratio[i] > 10:
            center[i] = np.inf
    return center

In [17]:
def get_focal_length(folder, img_name):
    if 'heic' in img_name:
        image = pyheif.read_heif(open(os.path.join(folder, img_name), 'rb'))
        if image.metadata:
            for metadata in image.metadata:
                if metadata['type'] == 'Exif':
                    img_file = io.BytesIO(metadata['data'][6:])
    else:
        img_file = open(os.path.join(folder, img_name), 'rb')

    output = exifread.process_file(img_file, details=False)
    focal_length = output['EXIF FocalLengthIn35mmFilm'].values[0]
    return focal_length


def model_for_bbox(data, img_names, folder, draw_img=False):
    data_res = []
    
    for img, img_name in tqdm(zip(data, img_names), total=len(img_names)):
        results = get_plates_from_image(img)[1]
        
        if results != []:
            index_bbox = central_bbox(results, img.shape[1] / 2)
            if any([i != np.inf for i in index_bbox]):
                index_center = np.argmin(index_bbox)
                focal_length = get_focal_length(folder, img_name)
                data_res.append([img_name] + results[index_center] + [focal_length])

            if draw_img == True:
                img = img.copy()
                img = Image.fromarray(img)
                draw = ImageDraw.Draw(img)
                print(img_name)
                for i, bbox in enumerate(results):
                    # print(bbox[0])
                    if i == np.argmin(index_bbox) and i != np.inf:
                        draw.rectangle([(bbox[0], bbox[1]), (bbox[2], bbox[3])], outline='green', width=2)
                    else:
                        draw.rectangle([(bbox[0], bbox[1]), (bbox[2], bbox[3])], outline='red', width=2)
                img.thumbnail((320, 320))
                display(img)
            
        else:
            print(img_name)
            
        del img

    return data_res

In [18]:
train_data_res = model_for_bbox(train_data, train_img_names, '/train/') #181

  0%|          | 0/530 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [19]:
test_data_res = model_for_bbox(test_data, test_img_names, '/test/')

  0%|          | 0/521 [00:00<?, ?it/s]

img_2674.heic


In [20]:
def df_split(data_res, labels_df=None):
    data_df = pd.DataFrame(data_res, columns = ['image_name', 'x_min', 'y_min', 'x_max', 'y_max', 'focal_length'])
    data_df['wigth'] = data_df['x_max'] - data_df['x_min']
    data_df['1/x'] = 1 / data_df['wigth']
    data_df['focal_length/x'] = data_df['focal_length'] / data_df['wigth']

    if labels_df is None:        
        data_df = data_df.dropna()
        data_df_X = data_df[['focal_length/x']]
        return data_df, data_df_X
    else:
        data_df = pd.merge(labels_df, data_df, how='left')
        data_df = data_df.dropna()
        data_df_y = data_df['distance']
        data_df_X = data_df[['focal_length/x']]

        return data_df, data_df_X, data_df_y

In [21]:
train_data_df, train_data_df_X, train_data_df_y = df_split(train_data_res, train_labels_df)

In [22]:
test_data_df, test_data_df_X = df_split(test_data_res)

In [23]:
model_2 = LinearRegression()
y_pred = cross_val_predict(model_2, train_data_df_X, train_data_df_y)
r2_score(train_data_df_y, y_pred)

0.9455373272008265

In [24]:
model_2 = LinearRegression()

# Fit model
model_2.fit(train_data_df_X, train_data_df_y.values)

LinearRegression()

In [25]:
def predict_and_save(name, test_data = test_data_df_X[['focal_length/x']]):

    test_data_df['distance'] = model_2.predict(test_data)
    solution_df = test_data_df[['image_name', 'distance']]

    lost_test_items = []
    mean_lost = test_data_df['distance'].mean()
    for file_name in set(test_img_names) - set(solution_df['image_name'].values):
        lost_test_items.append([file_name, mean_lost])
    lost_test_items_df = pd.DataFrame(lost_test_items, columns=['image_name', 'distance'])

    solution_df = pd.concat([solution_df, lost_test_items_df])
    solution_df.to_csv(name + '.csv', sep=';', index=False)

In [26]:
predict_and_save('solution')